# Imports and Setup
- If I am doing any processing of files in a folder I like to import os and glob
- import cellpose models for training of inferance
- import something to read your images 
- import libraries to save your results


In [1]:
import os
import glob
import sys

from cellpose import models
import napari
import torch
import nd2
import numpy as np

import skimage as ski

import sutils

Create the napari viewer to visualize images and mask

In [2]:
viewer = napari.Viewer()

Find the image files to work with. Different path for Window, Mac, and Linux

In [3]:

x = ski.io.imread("files/cellpose/Data/01-yeast_many.tif")
x.shape

(2048, 2048)

Add the image to the napari viewer

In [4]:
viewer.layers.clear()
viewer.add_image(x)

<Image layer 'x' at 0x28e49554100>

## Setup the cellpose model



## Simple way - On Workstation

#### Model
- cellpose has a number of pretrained models to use. Start with `cyto` or `cyto2` (even it is nuclei)
- info about pretrained cellpose [models](https://cellpose.readthedocs.io/en/latest/models.html)

In [5]:
model = models.Cellpose(gpu=True, model_type='cyto2')

## Flexible way - On Workstation or Mac

Cellpose runs much faster on a gpu, and normally the only alternative is to run it on a cpu.  BUT there is a way to take advantage of the MacOS gpu that is a bit of a compromise.  If you are running on your own laptop you will want to use this instead.

#### GPU setting
- on windows or linux with an NVIDIA GPU, set `gpu=True``
- on M1 or M2 mac, set `device=torch.device('mps')`
- on old Mac or Windows without GPU, set `gpu=False` -- this will be slower



In [6]:
if sys.platform == 'darwin':
    d = torch.device('mps')
    model = models.Cellpose(gpu=False, device=d, model_type='cyto2')
else:
    # change gpu=True if on windows, and get rid of device
    model = models.Cellpose(gpu=True, model_type='cyto2')

# `model.eval`
Parameters
----------

The basic options:
- x : the image, can be a 2d numpy array, a list of numpy arrays,
or a 3d numpy array
- diameter : The approximate size of the object you are trying to segment
- channels :
    - [0, 0] for a grayscale image 

Returns
-------
- masks : An array or list of arrays with segmenation labels/masks
- flows : A list of numpy arrays with fow
- diams:  The diameter used

## Try cellpose with no parameter changes

In [7]:

masks, flows, styles, diams = model.eval(x, channels=[0, 0])
diams

30.0

masks is the label image we are interested in, the others we will talk about later

Show the results from cellpose a layers in napari

In [8]:
viewer.layers.clear()
viewer.add_image(x)
viewer.add_labels(masks, name='default diameter (30)')


<Labels layer 'default diameter (30)' at 0x28e49543a60>

## diameter

See what happens when the diameter is changed. What diameter is the best?

In [9]:

masks, flows, styles, diams = model.eval(x, channels=[0, 0],
                                         diameter=45)

viewer.add_labels(masks, name='diameter 45')

<Labels layer 'diameter 45' at 0x28e4bb9f190>

In [10]:

masks, flows, styles, diams = model.eval(x, channels=[0, 0],
                                         diameter=15)

viewer.add_labels(masks, name='diameter 15')

<Labels layer 'diameter 15' at 0x28e3e179390>

## flow_threshold and cellprob_threshold

In [11]:
model.eval?

Signature:
model.eval(
    x,
    batch_size=8,
    channels=None,
    channel_axis=None,
    z_axis=None,
    invert=False,
    normalize=True,
    diameter=30.0,
    do_3D=False,
    anisotropy=None,
    net_avg=False,
    augment=False,
    tile=True,
    tile_overlap=0.1,
    resample=True,
    interp=True,
    flow_threshold=0.4,
    cellprob_threshold=0.0,
    min_size=15,
    stitch_threshold=0.0,
    rescale=None,
    progress=None,
    model_loaded=False,
)
Docstring:
run cellpose and get masks

Parameters
----------
x: list or array of images
    can be list of 2D/3D images, or array of 2D/3D images, or 4D image array

batch_size: int (optional, default 8)
    number of 224x224 patches to run simultaneously on the GPU
    (can make smaller or bigger depending on GPU memory usage)

channels: list (optional, default None)
    list of channels, either of length 2 or of length number of images by 2.
    First element of list is the channel to segment (0=grayscale, 1=red, 2=green,

We also have cellprob_threshold and flow_threshold we can tweak.

flow_threshold:  defaults to 0.4, range is 0->3.  Higher values let more objects through, where the shape is less solidly known.

cellprob_threshold:  defaults to 0.0, range is -10->10.  Lower values let more of an existing object through, so objects end up larger.  At high values will start to remove objects.

<img src="files/cellpose/cellpose_params.png" width="950">

In [12]:
masks, flows, styles, diams = model.eval(x, channels=[0, 0],
                                             diameter=45,
                                             cellprob_threshold=-6)
viewer.add_labels(masks, name='diameter 45, cellprob -6')

masks, flows, styles, diams = model.eval(x, channels=[0, 0],
                                                diameter=45,
                                                cellprob_threshold=6)
viewer.add_labels(masks, name='diameter 45, cellprob 6')

<Labels layer 'diameter 45, cellprob 6' at 0x28e3eb6a110>